In [1]:
import numpy as np
import pandas as pd
from tpot import TPOTRegressor
import importlib
from utils import *

In [2]:
data_dir = 'qsar/'
label = 'Act'
random_state = 1618
# my_datasets = pd.read_csv('qsar_datasets.txt', header = None)[0].values.tolist()
my_datasets = ['DPP4', 'HIVINT', 'HIVPROT', 'METAB', 'NK1', 'OX1', 'OX2', 'PGP', 'PPB']

In [3]:
def get_predictions(dat_name, random_state):
    file_name = dat_name + '_'  + str(random_state)
    # Read in the data:
    train_data = pd.read_csv(data_dir + dat_name + '_training_preprocessed.csv', index_col = 'MOLECULE')
    test_data = pd.read_csv(data_dir + dat_name + '_test_preprocessed.csv', index_col = 'MOLECULE')
    testing_features = test_data.drop([label], axis=1)
    training_features = train_data.drop([label], axis=1)
    training_target = train_data[label]
    testing_target = test_data[label]

    # Run the TPOT recommended pipeline
    pipe_file = 'pipelines.' + dat_name + '_' + str(random_state)
    
    pipe = importlib.import_module(pipe_file, package=None)

    tpot_results = pipe.opt_pipe(training_features, testing_features)
    exported_pipeline = tpot_results['pipe']
    training_features = tpot_results['train_feat'] # in case imputation was done
    testing_features = tpot_results['test_feat'] # in case imputation was done

    tpot_obj= TPOTRegressor()
    
    # Set random_state:
    if hasattr(exported_pipeline, 'steps'): # if it is a pipeline
        tpot_obj._set_param_recursive(exported_pipeline.steps, 'random_state', random_state)
    elif hasattr(exported_pipeline, 'random_state'): # if this is a classifier
        exported_pipeline.set_params(**random_state)

    # Fit the exported pipeline to the training:
    model = exported_pipeline.fit(training_features, training_target)
    
    # Get predictions:
    pred = exported_pipeline.predict(testing_features)
    predictions = pd.DataFrame({'MOLECULE': list(test_data.index), 'Act_pred': pred, 'Act': testing_target})
    predictions.to_csv('predictions/' + file_name + '_test.csv')
    
    return({'Dataset': dat_name,
            'TPOT score': exported_pipeline.score(testing_features, testing_target),
            'Testing R^2': np.corrcoef(pred, testing_target)[0,1]**2})


In [4]:
# my_datasets = ['METAB', 'HIVINT']
mtypes = {'datasets': my_datasets,
          'seed': [random_state]}
mtype_grid = expand_grid(mtypes) # data type grid

results = mtype_grid.apply(
    lambda r: get_predictions(r.datasets, r.seed), 
    axis = 1, result_type = 'expand')
results.to_csv('MLP_results/' + str(random_state) + ".csv")

/Users/ttle/anaconda3/envs/tpotDataSel/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/ttle/anaconda3/envs/tpotDataSel/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [5]:
results

,Dataset,TPOT score,Testing R^2
0,DPP4,0.025031,0.254206
1,HIVINT,0.275124,0.302616
2,HIVPROT,0.094245,0.526974
3,METAB,0.647080,0.663476
4,NK1,0.239584,0.393388
5,OX1,0.585474,0.591423
6,OX2,0.558934,0.559489
7,PGP,0.565596,0.575390
8,PPB,0.478269,0.521297
